In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), "ucla-covid19-prediction")
data_dir

'/home/kien/Documents/school/cs-145-data-mining/ucla-covid19-prediction'

In [4]:
def transform_date(x):
    x = x.split("-")
    return "-".join([x[-1], x[0], x[1]])

In [5]:
# Class dataset
dataset = pd.read_csv(os.path.join(data_dir, "train_trendency.csv"), index_col=0)
dataset["Date"] = dataset["Date"].apply(transform_date)
dataset

,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate
0,Alabama,2021-01-12,407848,5573,211684.0,190591.0,8318.021857,1981019.0,1.366440,40402.697430
1,Alaska,2021-01-12,50544,225,7165.0,43132.0,6906.205360,1357708.0,0.445351,185594.597735
2,Arizona,2021-01-12,636100,10482,86757.0,538861.0,8739.177523,5742443.0,1.647854,42806.225328
3,Arkansas,2021-01-12,259553,4121,229700.0,25732.0,8600.724235,2208736.0,1.587730,73190.174047
4,California,2021-01-12,2832085,31345,NaN,2773506.0,7098.216165,36508384.0,1.110900,92397.696784
...,...,...,...,...,...,...,...,...,...,...
3945,Virginia,2021-03-31,618976,10252,NaN,NaN,7251.767584,6466133.0,1.656284,75755.592601
3946,Washington,2021-03-31,364486,5247,NaN,NaN,4786.488792,5890924.0,1.439561,77360.561731
3947,West Virginia,2021-03-31,141738,2676,NaN,NaN,7908.837835,2468123.0,1.887991,137718.780881
3948,Wisconsin,2021-03-31,636041,7315,NaN,NaN,10923.970972,7585402.0,1.150083,130278.883367


In [6]:
test_dataset = pd.read_csv(os.path.join(data_dir, "test.csv"), index_col=0)
test_dataset["Date"] = test_dataset["Date"].apply(transform_date)
test_dataset = test_dataset.drop(columns=["Confirmed", "Deaths"])
test_dataset

,Province_State,Date
0,Alabama,2021-04-01
1,Alaska,2021-04-01
2,Arizona,2021-04-01
3,Arkansas,2021-04-01
4,California,2021-04-01
...,...,...
1495,Virginia,2021-04-30
1496,Washington,2021-04-30
1497,West Virginia,2021-04-30
1498,Wisconsin,2021-04-30


# Preprocessing

In [7]:
drop_columns = ["Recovered",
                "Active",
                "Incident_Rate",
                "Total_Test_Results",
                "Case_Fatality_Ratio",
                "Testing_Rate"
               ]

def to_log(x):
    return np.log(x + 1)

def to_exp(x):
    return np.exp(x) - 1

def transform_dataset(dataset):
    dataset['LogConfirmed'] = to_log(dataset["Confirmed"])
    dataset['LogFatalities'] = to_log(dataset["Deaths"])
    dataset = dataset.drop(columns=drop_columns)
    return dataset

In [8]:
dataset = transform_dataset(dataset)
dataset.head()

,Province_State,Date,Confirmed,Deaths,LogConfirmed,LogFatalities
0,Alabama,2021-01-12,407848,5573,12.918652,8.625868
1,Alaska,2021-01-12,50544,225,10.830619,5.420535
2,Arizona,2021-01-12,636100,10482,13.363113,9.257510
3,Arkansas,2021-01-12,259553,4121,12.466720,8.324094
4,California,2021-01-12,2832085,31345,14.856524,10.352842


# Heuristic Approach

In [9]:
dfs = []
for loc, df in tqdm(dataset.groupby('Province_State')):
    df = df.sort_values(by='Date')
    df['LogFatalities'] = df['LogFatalities'].cummax()
    df['LogConfirmed'] = df['LogConfirmed'].cummax()
    df['LogConfirmedNextDay'] = df['LogConfirmed'].shift(-1)
    df['DateNextDay'] = df['Date'].shift(-1)
    df['LogFatalitiesNextDay'] = df['LogFatalities'].shift(-1)
    df['LogConfirmedDelta'] = df['LogConfirmedNextDay'] - df['LogConfirmed']
    df['LogFatalitiesDelta'] = df['LogFatalitiesNextDay'] - df['LogFatalities']
    dfs.append(df)

dfs = pd.concat(dfs)

100%|██████████| 50/50 [00:00<00:00, 544.01it/s]


In [10]:
dfs.shape
dfs.tail()

,Province_State,Date,Confirmed,Deaths,LogConfirmed,LogFatalities,LogConfirmedNextDay,DateNextDay,LogFatalitiesNextDay,LogConfirmedDelta,LogFatalitiesDelta
3749,Wyoming,2021-03-27,56046,695,10.933946,6.545350,10.933946,2021-03-28,6.545350,0.000000,0.000000
3799,Wyoming,2021-03-28,56046,695,10.933946,6.545350,10.936512,2021-03-29,6.545350,0.002566,0.000000
3849,Wyoming,2021-03-29,56190,695,10.936512,6.545350,10.937330,2021-03-30,6.545350,0.000818,0.000000
3899,Wyoming,2021-03-30,56236,695,10.937330,6.545350,10.938645,2021-03-31,6.552508,0.001315,0.007158
3949,Wyoming,2021-03-31,56310,700,10.938645,6.552508,NaN,NaN,NaN,NaN,NaN


In [11]:
deltas = dfs[dfs.LogConfirmed > 0].dropna().sort_values(by='LogConfirmedDelta', ascending=False)
deltas = deltas[deltas["Date"] >= "2021-03-16"]

confirmed_deltas = pd.concat([
    deltas.groupby("Province_State")[['LogConfirmedDelta']].mean(),
    deltas.groupby("Province_State")[['LogConfirmedDelta']].std(),
    deltas.groupby("Province_State")[['LogConfirmedDelta']].count(),
    deltas.groupby("Province_State")[['LogConfirmed']].max()
], axis=1)

confirmed_deltas.columns = ["avg", "std", "cnt", "max"]
confirmed_deltas.to_csv('confirmed_deltas.csv')

confirmed_deltas.tail(10)

,avg,std,cnt,max
Province_State,,,,
South Dakota,0.001702,0.000572,15,11.674160
Tennessee,0.001470,0.001238,15,13.605444
Texas,0.001306,0.000413,15,14.841347
Utah,0.001144,0.000362,15,12.861331
Vermont,0.008189,0.003001,15,9.857967
Virginia,0.002394,0.000517,15,13.334150
Washington,0.002493,0.001169,15,12.802808
West Virginia,0.002746,0.000538,15,11.858803
Wisconsin,0.001022,0.000529,15,13.361876


# Confirmed Cases

In [12]:
DECAY = 0.93

In [13]:
confirmed_deltas = dataset.groupby('Province_State').count()
confirmed_deltas['DELTA'] = 0.18

In [14]:
daily_log_confirmed = dfs.pivot('Province_State', 'Date', 'LogConfirmed').reset_index()
daily_log_confirmed = daily_log_confirmed.sort_values('2021-03-15', ascending=False)
daily_log_confirmed.to_csv('daily_log_confirmed.csv', index=False)

DATEFORMAT = '%Y-%m-%d'

for i, d in tqdm(enumerate(pd.date_range('2021-04-01', '2021-04-30'))):
    new_day = str(d).split(' ')[0]
    last_day = dt.datetime.strptime(new_day, DATEFORMAT) - dt.timedelta(days=1)
    last_day = last_day.strftime(DATEFORMAT)
    for loc in confirmed_deltas.index:
        confirmed_delta = confirmed_deltas.loc[confirmed_deltas.index == loc, 'DELTA'].values[0]
        daily_log_confirmed.loc[daily_log_confirmed["Province_State"] == loc, new_day] = daily_log_confirmed.loc[daily_log_confirmed["Province_State"] == loc, last_day] + \
            confirmed_delta * DECAY ** i

30it [00:00, 32.80it/s]


In [15]:
daily_log_confirmed.head(10)

Date,Province_State,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2021-01-19,2021-01-20,...,2021-04-21,2021-04-22,2021-04-23,2021-04-24,2021-04-25,2021-04-26,2021-04-27,2021-04-28,2021-04-29,2021-04-30
4,California,14.856524,14.869859,14.883679,14.896656,14.908490,14.918921,14.928356,14.939970,14.946149,...,17.126624,17.165835,17.202302,17.236216,17.267756,17.297088,17.324367,17.349737,17.373331,17.395273
42,Texas,14.515954,14.528829,14.542401,14.555027,14.563460,14.570276,14.575471,14.584442,14.597380,...,16.853500,16.892711,16.929178,16.963092,16.994632,17.023965,17.051244,17.076613,17.100207,17.122149
8,Florida,14.223295,14.232557,14.241558,14.252518,14.260316,14.267401,14.272481,14.278677,14.286146,...,16.548380,16.587592,16.624058,16.657972,16.689513,16.718845,16.746124,16.771493,16.795087,16.817029
31,New York,13.967856,13.980404,13.992362,14.008544,14.021604,14.032686,14.043420,14.053498,14.062224,...,16.454389,16.493601,16.530068,16.563982,16.595522,16.624854,16.652133,16.677503,16.701096,16.723038
12,Illinois,13.854894,13.860514,13.866853,13.873142,13.878174,13.882075,13.885237,13.889256,13.893725,...,16.045507,16.084719,16.121186,16.155100,16.186640,16.215972,16.243251,16.268621,16.292215,16.314157
9,Georgia,13.558930,13.569980,13.581464,13.593780,13.604375,13.612165,13.618221,13.627471,13.637161,...,15.884617,15.923829,15.960295,15.994209,16.025750,16.055082,16.082361,16.107730,16.131324,16.153266
34,Ohio,13.583502,13.591917,13.601443,13.610260,13.618897,13.625264,13.630466,13.636444,13.644051,...,15.844188,15.883400,15.919866,15.953781,15.985321,16.014653,16.041932,16.067302,16.090895,16.112837
37,Pennsylvania,13.511692,13.522485,13.532040,13.540078,13.548271,13.555425,13.560059,13.569544,13.576727,...,15.854076,15.893288,15.929755,15.963669,15.995209,16.024541,16.051820,16.077190,16.100784,16.122726
32,North Carolina,13.362916,13.370900,13.386153,13.399754,13.411784,13.421932,13.421932,13.436441,13.445769,...,15.736994,15.776206,15.812673,15.846587,15.878127,15.907459,15.934738,15.960108,15.983701,16.005644
29,New Jersey,13.295896,13.309059,13.320758,13.331056,13.340840,13.349056,13.355180,13.362487,13.371005,...,15.731162,15.770374,15.806840,15.840755,15.872295,15.901627,15.928906,15.954276,15.977869,15.999811


# Deaths

In [16]:
death_deltas = dfs[dfs["Deaths"] > 0].dropna().sort_values(by='LogFatalitiesDelta', ascending=False)
death_deltas

,Province_State,Date,Confirmed,Deaths,LogConfirmed,LogFatalities,LogConfirmedNextDay,DateNextDay,LogFatalitiesNextDay,LogConfirmedDelta,LogFatalitiesDelta
660,Hawaii,2021-01-25,26019,342,10.166621,5.837730,10.167043,2021-01-26,5.996452,0.000423,0.158722
351,Alaska,2021-01-19,52243,230,10.863680,5.442418,10.866929,2021-01-20,5.537334,0.003249,0.094917
3216,Kentucky,2021-03-17,418372,5056,12.944129,8.528529,12.945984,2021-03-18,8.613412,0.001855,0.084883
2947,West Virginia,2021-03-11,134496,2338,11.809297,7.757479,11.811866,2021-03-12,7.828835,0.002569,0.071356
1001,Alaska,2021-02-01,54457,262,10.905185,5.572154,10.907734,2021-02-02,5.634790,0.002549,0.062636
...,...,...,...,...,...,...,...,...,...,...,...
3890,South Dakota,2021-03-30,117495,1935,11.674160,7.568379,11.676404,2021-03-31,7.568379,0.002244,0.000000
1917,Louisiana,2021-02-19,422287,9440,12.953443,9.152817,12.953443,2021-02-20,9.152817,0.000000,0.000000
1717,Louisiana,2021-02-15,420394,9325,12.948950,9.140561,12.948950,2021-02-16,9.140561,0.000000,0.000000
1567,Louisiana,2021-02-12,418585,9276,12.944638,9.135293,12.944638,2021-02-13,9.135293,0.000000,0.000000


In [17]:
loc_confirmed_death_deltas = pd.concat([
    death_deltas.groupby('Province_State')[['LogFatalitiesDelta']].mean(),
    death_deltas.groupby('Province_State')[['LogFatalitiesDelta']].std(),
    death_deltas.groupby('Province_State')[['LogFatalitiesDelta']].count(),
    death_deltas.groupby('Province_State')[['LogFatalities']].max()
], axis=1)
loc_confirmed_death_deltas.columns = ['avg', 'std', 'cnt', 'max']
loc_confirmed_death_deltas = loc_confirmed_death_deltas[loc_confirmed_death_deltas.cnt >= 3]

loc_confirmed_death_deltas.sort_values(by='avg').head(10)
loc_confirmed_death_deltas.sort_values(by='avg').tail(10)
loc_confirmed_death_deltas.to_csv('loc_confirmed_death_deltas.csv')

In [18]:
death_deltas = dataset.groupby('Province_State').count()
death_deltas['DELTA'] = 0.15

In [19]:
daily_log_deaths = dfs.pivot('Province_State', 'Date', 'LogFatalities').reset_index()
daily_log_deaths = daily_log_deaths.sort_values('2021-03-15', ascending=False)
daily_log_deaths.to_csv('daily_log_deaths.csv', index=False)

DATEFORMAT = '%Y-%m-%d'

for i, d in tqdm(enumerate(pd.date_range('2021-04-01', '2021-04-30'))):
    new_day = str(d).split(' ')[0]
    last_day = dt.datetime.strptime(new_day, DATEFORMAT) - dt.timedelta(days=1)
    last_day = last_day.strftime(DATEFORMAT)
    for loc in death_deltas.index:
        death_delta = death_deltas.loc[death_deltas.index == loc, 'DELTA'].values[0]
        daily_log_deaths.loc[daily_log_deaths["Province_State"] == loc, new_day] = daily_log_deaths.loc[
            daily_log_deaths["Province_State"] == loc, last_day] + \
            death_delta * DECAY ** i

30it [00:00, 33.52it/s]


In [20]:
daily_log_deaths.head(10)

Date,Province_State,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2021-01-19,2021-01-20,...,2021-04-21,2021-04-22,2021-04-23,2021-04-24,2021-04-25,2021-04-26,2021-04-27,2021-04-28,2021-04-29,2021-04-30
4,California,10.352842,10.369483,10.389549,10.410094,10.421537,10.428541,10.432998,10.453255,10.471015,...,12.665995,12.698672,12.729061,12.757322,12.783606,12.808050,12.830782,12.851923,12.871585,12.889870
31,New York,10.597160,10.601448,10.607476,10.612459,10.616609,10.621181,10.625562,10.630287,10.635567,...,12.501813,12.534489,12.564878,12.593140,12.619423,12.643867,12.666599,12.687741,12.707402,12.725687
42,Texas,10.338382,10.350670,10.363883,10.376206,10.386808,10.392098,10.395497,10.405262,10.419062,...,12.460266,12.492942,12.523331,12.551593,12.577876,12.602320,12.625052,12.646194,12.665855,12.684140
8,Florida,10.053114,10.060363,10.069595,10.077441,10.086017,10.091543,10.097202,10.103854,10.109648,...,12.093142,12.125818,12.156207,12.184469,12.210752,12.235196,12.257928,12.279070,12.298731,12.317016
37,Pennsylvania,9.800679,9.820812,9.836867,9.848820,9.859745,9.865578,9.869465,9.874265,9.894043,...,11.805679,11.838355,11.868744,11.897006,11.923290,11.947733,11.970466,11.991607,12.011268,12.029553
29,New Jersey,9.905486,9.911555,9.911555,9.919410,9.924025,9.925249,9.926178,9.928814,9.936197,...,11.785008,11.817685,11.848074,11.876335,11.902619,11.927062,11.949795,11.970936,11.990598,12.008883
12,Illinois,9.878067,9.884203,9.889642,9.897168,9.904537,9.906034,9.909420,9.911158,9.917686,...,11.744207,11.776883,11.807272,11.835534,11.861817,11.886261,11.908993,11.930135,11.949796,11.968081
34,Ohio,9.629248,9.638480,9.645882,9.654770,9.662371,9.669788,9.677653,9.683900,9.689923,...,11.529982,11.562658,11.593047,11.621309,11.647593,11.672036,11.694769,11.715910,11.735571,11.753856
9,Georgia,9.364177,9.376194,9.390660,9.404179,9.416704,9.417111,9.422302,9.439307,9.455558,...,11.531190,11.563866,11.594255,11.622517,11.648800,11.673244,11.695976,11.717118,11.736779,11.755064
21,Michigan,9.567805,9.570599,9.582731,9.585415,9.593560,9.593560,9.594718,9.595807,9.600421,...,11.424055,11.456732,11.487120,11.515382,11.541666,11.566109,11.588842,11.609983,11.629644,11.647929


# Create submission file

In [21]:
submission = pd.read_csv(os.path.join(data_dir, "submission.csv"), index_col=0)

In [22]:
submission.head()

,Confirmed,Deaths
ID,,
0,-1,-1
1,-1,-1
2,-1,-1
3,-1,-1
4,-1,-1


In [23]:
confirmed = []
fatalities = []
for loc, d in tqdm(test_dataset.values):
    c = to_exp(daily_log_confirmed.loc[daily_log_confirmed["Province_State"] == loc, d].values[0])
    f = to_exp(daily_log_deaths.loc[daily_log_deaths["Province_State"] == loc, d].values[0])

    if d != "2021-04-01":
        last_day = dt.datetime.strptime(d, DATEFORMAT) - dt.timedelta(days=1)
        last_day = last_day.strftime(DATEFORMAT)
        c += to_exp(daily_log_confirmed.loc[daily_log_confirmed["Province_State"] == loc, last_day].values[0])
        f += to_exp(daily_log_deaths.loc[daily_log_deaths["Province_State"] == loc, last_day].values[0])
        
    confirmed.append(c)
    fatalities.append(f)
        

100%|██████████| 1500/1500 [00:00<00:00, 1790.88it/s]


In [24]:
submission['Confirmed'] = confirmed
submission['Deaths'] = fatalities
submission

,Confirmed,Deaths
ID,,
0,6.170317e+05,12262.160432
1,7.530637e+04,363.815952
2,1.007831e+06,19713.003431
3,3.955584e+05,6536.641284
4,4.392285e+06,68866.724738
...,...,...
1495,1.197069e+07,135846.953112
1496,7.048987e+06,69532.312487
1497,2.741156e+06,35467.389201


In [25]:
submission.iloc[0]

Confirmed    617031.659562
Deaths        12262.160432
Name: 0, dtype: float64

In [26]:
submission.iloc[50]

Confirmed    1.346503e+06
Deaths       2.636010e+04
Name: 50, dtype: float64

In [27]:
submission.to_csv("team2.csv")